In [3]:
import random
import math

def ant_colony_routing(distance, num_ants, num_iterations, alpha=1, beta=2, rho=0.5, tau0=1.0, start=0, end=4):
    n = len(distance)
    
    tau = [[tau0 for _ in range(n)] for _ in range(n)]
    eta = [[0 if i == j else 1 / distance[i][j] if distance[i][j] != 0 else 0 for j in range(n)] for i in range(n)]

    best_path = None
    best_length = math.inf

    for _ in range(num_iterations):
        paths = []
        lengths = []

        # Step 1: Each ant builds a path
        for k in range(num_ants):
            path = construct_path(distance, tau, eta, alpha, beta, start, end)
            length = path_length(path, distance)
            paths.append(path)
            lengths.append(length)

            if length < best_length:
                best_length = length
                best_path = path

        # Step 2: Evaporate pheromone
        for i in range(n):
            for j in range(n):
                tau[i][j] *= (1 - rho)

        # Step 3: Update pheromone based on good paths
        for k in range(num_ants):
            for i in range(len(paths[k]) - 1):
                a, b = paths[k][i], paths[k][i + 1]
                tau[a][b] += 1 / lengths[k]
                tau[b][a] = tau[a][b]  # symmetric network

    return best_path, best_length


def construct_path(distance, tau, eta, alpha, beta, start, end):
    n = len(distance)
    current = start
    visited = {start}
    path = [start]

    while current != end:
        allowed = [i for i in range(n) if i not in visited and distance[current][i] != 0]

        # If stuck, end prematurely
        if not allowed:
            break

        # Calculate probabilities
        probs = []
        for j in allowed:
            probs.append((tau[current][j] ** alpha) * (eta[current][j] ** beta))
        total = sum(probs)
        probs = [p / total for p in probs]

        # Roulette wheel selection
        next_node = random.choices(allowed, weights=probs, k=1)[0]
        path.append(next_node)
        visited.add(next_node)
        current = next_node

    
    if path[-1] != end:
        path.append(end)
    return path


# Function to calculate total distance of a path
def path_length(path, distance):
    total = 0
    for i in range(len(path) - 1):
        total += distance[path[i]][path[i + 1]]
    return total


network = [
    [0, 2, 0, 1, 0],
    [2, 0, 3, 2, 0],
    [0, 3, 0, 0, 1],
    [1, 2, 0, 0, 3],
    [0, 0, 1, 3, 0]
]


num_ants = 10
num_iterations = 50
alpha = 1     # influence of pheromone
beta = 3      # influence of distance
rho = 0.4     # evaporation rate

best_path, best_distance = ant_colony_routing(network, num_ants, num_iterations, alpha, beta, rho, start=0, end=4)

print("\n=== Ant Colony Optimization for Network Routing ===")
print("Best Route Found :", best_path)
print("Total Distance   :", best_distance)



=== Ant Colony Optimization for Network Routing ===
Best Route Found : [0, 3, 4]
Total Distance   : 4
